In [ ]:
#-*- coding: utf-8 -*- 

In [ ]:
import os, sys 
from google.colab import drive 
drive.mount('/content/mnt') 
nb_path = '/content/notebooks' 
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path) 
sys.path.insert(0, nb_path)

Mounted at /content/mnt


In [ ]:
import os

import pandas as pd
import numpy as np

from pathlib import Path
from datetime import datetime

BASE_DIR = Path(r"/content/mnt/Shared drives/MBTI/")
data_list = os.listdir(BASE_DIR / "full_data")
print(data_list[:5])

['MBTI_add.csv', 'ISTJ.csv', 'ESTJ.csv', 'ISFJ.csv', 'ESFJ.csv']


In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]


In [ ]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

In [ ]:
mbti_list = pd.read_csv(BASE_DIR / "full_data"/"MBTI_add.csv")

In [ ]:
mbti_list

,type,name,link
0,ISTJ,[MBTI 성격유형 4편] ISTJ - 논리주의 철저한 조사관,https://www.youtube.com/watch?v=5oKctIz6vhE
1,ESTJ,[MBTI 성격유형 5편] ESTJ - 실용주의 엄격한 관리자,https://www.youtube.com/watch?v=-8UcVl7cl8o
2,ISFJ,[MBTI 성격유형 6편] ISFJ - 배려킹 충실한 서포터,https://www.youtube.com/watch?v=3f5MFs-hyuw
3,ESFJ,[MBTI 성격유형 7편] ESFJ - 해피 바이러스 정통파 인싸,https://www.youtube.com/watch?v=G5NyGyJLAdo
4,ISTP,[MBTI 성격유형 8편] ISTP - 고독한 특전사 에이스,https://www.youtube.com/watch?v=iQH1HzIw5a8
...,...,...,...
142,ESTP,"ESTP 엣팁 짤, 댓글 모음",https://www.youtube.com/watch?v=SZkfO7IBbU0
143,ENTP,"ENTP 엔팁 짤, 댓글 모음 2",https://www.youtube.com/watch?v=rPo1a3NCFsE
144,ENFJ,"ENFJ 엔프제 짤, 댓글 모음",https://www.youtube.com/watch?v=JbunSul0qL0
145,INTP,"INTP 인팁 짤, 댓글 모음 2",https://www.youtube.com/watch?v=aXL_X8EvZiQ


In [ ]:
def crawl_youtube_page_html_sources(urls):
    html_sources = []

    for i in range(len(url)):
        driver = webdriver.Chrome("chromedriver",options=options)
        driver.get(urls[i])

        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(3.0)
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

            if new_page_height == last_page_height:
                break
            last_page_height = new_page_height

        elements = driver.find_elements_by_css_selector("#more-replies")
        for element in elements :
          driver.execute_script("arguments[0].click();", element)

        html_source = driver.page_source
        html_sources.append(html_source)
        print("OK")

        driver.quit()
    return html_sources

In [ ]:
def get_user_IDs_comments_likes(html_sources):
    my_dataframes = []
    for html in html_sources:
        
        soup = BeautifulSoup(html, 'lxml')
        youtube_user_IDs = soup.select('div#header-author > a > span')
        youtube_comments = soup.select('yt-formatted-string#content-text')
        youtube_likes = soup.select('span#vote-count-middle')
        
        str_youtube_userIDs = []
        str_youtube_comments = []
        str_youtube_likes = []

        for i in range(len(youtube_user_IDs)):
            str_tmp = str(youtube_user_IDs[i].text)
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('                ','')
            str_tmp = str_tmp.replace('          ','')
            str_youtube_userIDs.append(str_tmp)

            str_tmp = str(youtube_comments[i].text) 
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('               ', '')
            str_youtube_comments.append(str_tmp)

            str_tmp = str(youtube_likes[i].text) 
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('               ', '')
            str_youtube_likes.append(str_tmp)
            
        pd_data = {"ID":str_youtube_userIDs, "Comment":str_youtube_comments, "Likes":str_youtube_likes}

        youtube_pd = pd.DataFrame(pd_data)

        my_dataframes.append(youtube_pd)
        
    return my_dataframes

In [ ]:
def merge_and_save():
  my_dataframe = pd.DataFrame()
  for i in range(len(url)):
    my_dataframe = pd.concat([my_dataframe,my_dataframes[i]], ignore_index=True)
  display(my_dataframe)
  title = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…《\》]', '', MBTI)
  my_dataframe.to_csv(BASE_DIR/"full_data"/"{}.csv".format(title),index = False, encoding='utf-8-sig')

In [ ]:
mbti = pd.read_csv(BASE_DIR/"full_data"/'MBTI_add.csv')

types = mbti["type"].unique()
my_dataframe = pd.DataFrame()

In [ ]:
types

array(['ISTJ', 'ESTJ', 'ISFJ', 'ESFJ', 'ISTP', 'ESTP', 'ISFP', 'ESFP',
       'INFJ', 'ENFJ', 'INFP', 'ENFP', 'INTJ', 'ENTJ', 'INTP', 'ENTP'],
      dtype=object)

In [ ]:
from tqdm import tqdm

In [ ]:
for MBTI in tqdm(types[:2]):
  titles = mbti[mbti["type"]==MBTI].name.to_list()
  url = mbti[mbti["type"]==MBTI].link.to_list()

  html_sorces = crawl_youtube_page_html_sources(url)
  my_dataframes = get_user_IDs_comments_likes(html_sorces)
  merge_and_save()




  0%|          | 0/2 [00:00<?, ?it/s]

OK
OK
OK
OK
OK
OK
OK


,ID,Comment,Likes
0,박태호,ISTJ 손 들어라. 솔직히 인터넷 상에 덧글 달 때 네이버 맞춤법 검사기 돌려서 ...,874
1,ove L,손 그리고 남 맞춤법 틀린 거 보면 지적해주고 싶은데 또 그렇지는 못함,95
2,이서린,엇 싀빠빠 님 나 스토킹함?,20
3,국화빵,세상에 정말요...? (지나가던 INFP,8
4,청청,헐 틀린것같애두 그냥 넘어가는데!!엔프피♥,5
...,...,...,...
2201,yulee,노잼이라니 똑땅....ㅋㅋ 잇티제도 다뤄 주셔서 감사합니다.,1
2202,Yu Stella,회사라는 공간에서 게다가 상사의 감정을 왜 생각해줘야 할까요 ㅎㅎ저는 상사로서 있었...,3
2203,홍소소,가장 많다는데 다 어디있나.....,6
2204,얌얌,상사의 기분 신경 안쓰는 저.. 같이 일하는 팀원 한 분이 e로 시작하시는거 같은데...,0





 50%|█████     | 1/2 [07:18<07:18, 438.12s/it]

OK
OK
OK
OK
OK
OK
OK


,ID,Comment,Likes
0,bread coffee,Estj 댓글 웃긴게 본인들은 거의 없고 남이 궁금해서 들어온게 대부분이야 ㅋㅋㅋㅋㅋㅋㅋ,858
1,haslee,Infp밖에 없음ㅋㄱㅌㅋㄱㅌㄲㅋㅋㄱㅋㄱㅋ,133
2,주원,미친ㅋㅋㄱㅋㄲㅋㄱㅋㅋestj 내친그 반응이랑 개똑같네 ㅋㅋㄱㅋㄱㅋ,9
3,이윤지,ㅋㅋㅋㅋㅋㅋ,1
4,하양,오 미친 맞아요,2
...,...,...,...
1765,이거왜샀지,전 ISTJ인데 제가 봤을때 ESTJ가 ISTJ랑 다른게 확실히 에너지 넘치고 진짜...,1
1766,Jineun Hong,"칼날같은 말투 고치려는 마음을 갖게 설득하기 어렵겠죠? 개인적으로 estj, ent...",13
1767,반지희12,남편이 ENTP라서 알고싶어왔다가 제껏도 보러왔어요. 저는 ESTJ인데 일하면서 대...,0
1768,푸루루루,남자친구는 게임도 잘하는 사람 아니면 스트레스 받아서 저랑 같이 게임 안 한대요ㅜㅜ,5





100%|██████████| 2/2 [12:34<00:00, 377.15s/it]


In [ ]:
for MBTI in tqdm(types[2:4]):
  titles = mbti[mbti["type"]==MBTI].name.to_list()
  url = mbti[mbti["type"]==MBTI].link.to_list()

  html_sorces = crawl_youtube_page_html_sources(url)
  my_dataframes = get_user_IDs_comments_likes(html_sorces)
  merge_and_save()




  0%|          | 0/2 [00:00<?, ?it/s]

OK
OK
OK
OK
OK
OK
OK
OK


,ID,Comment,Likes
0,ᄋᄉᄋ,isfj 댓글 잘 안단다고 하는데 댓글을 애초에 안다는게 아니고 하고싶은말 ㅈㄴ ...,1.4K
1,ji won,ㅋㅋㅋㅋㅋㅋㅋ아 인정해요 한 번 쓸 때 자세히 보기 뜰 때까지 쓰다가 결국엔 삭제해...,89
2,KOREA 漫画man,쌉인정 ㅋㅋㅋㅋㅋㅋ 나도 그런적 많음,11
3,LP M,ㅋㅋㅋㅋㅋㅋㅋㅋ제 얘긴줄...;; 진짜 구구절절충ㅋㅋㅋ,30
4,감쟈,구구절절충 ㅠㅠㅠ남들이 뭐라안하는데 해명먼저 하고있음ㅋㅋㅋㅋㅋ쓰다가 현타오면 삭제,62
...,...,...,...
5536,김아은,남자 ST NT들은 사이코패쓰남쟈들 90% 사이코패쓰모두 망혼 하지말아요,0
5537,태연_714,남친이 isfj인 estp인데 여친인데제 의견만 폭팔해서 제가 리드하면서데이트 다닙...,1
5538,무네무네,ENTP 아니세요?,1
5539,박예린,갠덕으로 너무 안맞는 유형.... ㅠㅠ 너무 착하지만 잘 해주다가 하루아침에 변하고...,0





 50%|█████     | 1/2 [18:40<18:40, 1120.50s/it]

OK
OK
OK
OK
OK
OK
OK


,ID,Comment,Likes
0,서지은,혼자 있는거 좋아해..ㄹㅇ 즐김 재밌음. 근데 친구가 놀자고하면 ㅇㅏ.. 지금 딱 ...,1K
1,대마왕쿠쿠,개공감하고 갑니다 ㅋㅋㅋㅋ,24
2,조연우,공감이요 ㅋㅋㅋㅋㅋ,4
3,플론,ㅇㅈㅇㅈㅇㅈㅇㅈㅇ!!!!!!!! 앜ㅋㅋ 똑같아,1
4,박민제,"이거ㄹㅇ,,,",1
...,...,...,...
1605,Lee Sarah,친구분 esfj 에니어 2번인가보네요ㅎㅎ,0
1606,희원,조회수랑 댓글수랑 차이 많이나는거 봐 역시 enfj들은 댓글 잘안다는거 맞구나 내가...,0
1607,JENNIE YU,ENTP인데 남친이 ESFJ라 궁금해서 찾아와봤더니 정말 남친 그자체네요 캘린더 정...,0
1608,임국장 ASMR,미립 16가지 mbti영상에도 저에대한 댓글도 많이없었는데..그런거였군요!!,0





100%|██████████| 2/2 [23:33<00:00, 706.87s/it]


In [ ]:
for MBTI in tqdm(types[4:6]):
  titles = mbti[mbti["type"]==MBTI].name.to_list()
  url = mbti[mbti["type"]==MBTI].link.to_list()

  html_sorces = crawl_youtube_page_html_sources(url)
  my_dataframes = get_user_IDs_comments_likes(html_sorces)
  merge_and_save()




  0%|          | 0/2 [00:00<?, ?it/s]

OK
OK
OK
OK
OK
OK
OK
OK


,ID,Comment,Likes
0,JIN,카톡하다가 내가 전달하려는 정보가 조금이라도 헷갈리면 바로 검색해보고 말함,2.7K
1,16 19,JIN ㄹㅇ ㅋㅋㅋㅋㅋ 혹시 맞춤법이 틀리진 않았나 먼저 검색해봄,379
2,쥬노,와 완전 팩트 ㅇㅈ ㅋㅋㅋ,71
3,고양이풀,ㅋㅋㅋ그거 받고 뉴스에서 일어난 얘기 전해줄 때 해당 뉴스 링크 올려주거나 캡쳐해서...,143
4,"스치면 인연,스며들면 스폰지밥",JIN 와 레알 난데이거,3
...,...,...,...
7852,현민ᄋ,내가 잇팁이긴한테 딱히 잇팁인게 맘에 들진않음 ㅋㅋ 동족혐오있음,0
7853,샤넬인간,근데 istp여자는 흔하지가 않데 원래 이 유형은 남자가 많은 유형이고 여자자체가 ...,2
7854,kokeku ka,mbti 저거 최신 현대 심리학에서는 거의 미신 취급받는건데 저걸 아직도 신봉하면서...,2
7855,현홍,어쩌라고 그럼 안보면 되는거지 왜 여기와서 이래...,1





 50%|█████     | 1/2 [26:17<26:17, 1577.69s/it]

OK
OK
OK
OK
OK
OK
OK
OK


,ID,Comment,Likes
0,돌돌돌,누가 ESTP 아니랄까봐 끝까지 보고 요약한 사람 한명도 없고 요약만 찾고있네,1.4K
1,매미,돌돌돌 개웃기넼ㅋㅋㅋㅋ요약찯고잇엇는데,51
2,젠젠,앜ㅋㅋㅋㅋㅋㅋㅋㅋ,1
3,아기거미,오 저도 estp라서 들어오자마자 요약 찾고 있었음,48
4,리요다,ㅁㅌㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,3
...,...,...,...
2530,게임용 계정,내 친구 엣팁이었는데 맨날 시비걸고 비꼬고 거짓말 치고 페드립하고 때리고..그랬음....,0
2531,문채윤,엣팁입니당 그건 mbti 문제가 아니라 그 분 인성과 생각에 문제가 있는 것 같네요...,2
2532,문채윤,손절하려는데 계속 잡는 걸 보니 그냥 계정님이좋아서 장난을 치는 것 같은데 스트레스...,2
2533,랄랄라,여러분 안녕하세요 신입 엣팁입니당엔팁이었는데 엣팁으로 바꼈어여,1





100%|██████████| 2/2 [33:09<00:00, 994.60s/it] 


In [ ]:
mbti[mbti["name"]=="INFP 인프피 짤, 댓글 모음 11"]

In [ ]:
a[a["name"]=="INFP 인프피 짤, 댓글 모음 11"].name.to_list()

['INFP 인프피 짤, 댓글 모음 11']

In [ ]:
a[a["name"]=="INFP 인프피 짤, 댓글 모음 11"].link.to_list()

['https://www.youtube.com/watch?v=XMrNkGK948I']

In [ ]:
MBTI = "INFP_"

In [ ]:
titles = a[a["name"]=="INFP 인프피 짤, 댓글 모음 11"].name.to_list()
url = a[a["name"]=="INFP 인프피 짤, 댓글 모음 11"].link.to_list()

html_sorces = crawl_youtube_page_html_sources(url)
my_dataframes = get_user_IDs_comments_likes(html_sorces)
merge_and_save()

OK


,ID,Comment,Likes
0,김햇,해당 영상들은 각 유형에 대해 일반화시키려는 목적이 아닙니다.\r'어떤 유형은 이 ...,326
1,렛타,하지만 boy..너무 들어맞아서 일반화 시키지 않아도 돼는걸,6
2,빵덕,노래 제목 너무너무 궁금해요 ㅜㅜㅜㅜㅜㅜㅜㅜ,0
3,김햇,@빵덕 설명란에 적어두었습니다 😊,3
4,라룰,5:16 에 블로그를 시작하라고 나오지만 요즘은 n번방 사건같은 사이버 폭력일이 많...,1
...,...,...,...
1566,수아,ㄴ네? 저 죽일거에요?ㅜㅜㅜㅜㅜㅜㅜㅜㅜㅜㅜㅜ,0
1567,호우영,나 왜 하트 줘?,0
1568,우힛,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,0
1569,이지호,너나 쥬거!!!,0


In [ ]:
 for MBTI in tqdm(types[14:16]):
  titles = mbti[mbti["type"]==MBTI].name.to_list()
  url = mbti[mbti["type"]==MBTI].link.to_list()

  html_sorces = crawl_youtube_page_html_sources(url)
  my_dataframes = get_user_IDs_comments_likes(html_sorces)
  merge_and_save()




  0%|          | 0/2 [00:00<?, ?it/s]

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK


,ID,Comment,Likes
0,j j,인팁 특 동영상 틀어놓고 댓글 정주행 하느라 동영상 내용은 잘 안들어옴,2.5K
1,공일리오,아니 세상에... 저 자신인줄 알았네요,195
2,민경훈,난줄ㅋㅋㅋㅋㅋ,34
3,sᴘᴀᴄᴇ ᴄᴏᴡʙᴏɪ,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,11
4,다해,제가 지금 그러고 있네요,36
...,...,...,...
14635,꺄항항항항항항항항,나 인팁인데 감정적으로 힘들 때 걍 한숨 자고 일어나면 아무일 없던 것처럼 리셋됨그...,0
14636,river peaceful,ㄴㄴ 이건 케바케인듯. 감정을 버리진 않음 감정 소듕함 다만 그 감정 갖고 뭘 할건...,0
14637,About Time,그동안 인생을 살아오며 제일 강하게 공감했던부분0:13 0:49 0:56 1:42 ...,1
14638,배고픈맹수,몸을 움직여도 뛰기는 싫고 오히려 음악 들으면서 생각만 많아지는.. 음악에까지 속으...,0





 50%|█████     | 1/2 [58:24<58:24, 3504.36s/it]

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK


,ID,Comment,Likes
0,맥스,ㅋㅋㅋㅋ어디서 봤는데ENTP: 응응 그렇구나..사람마다 다 생각이 다르니까.. (물...,4.5K
1,외둘,맥스 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ아,43
2,라이,이건 댓글 안남길 수가 없다 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ미친 공감 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,286
3,Martinez,하씹ㅋㅋㅋㅋㅋㅋㅋㅋ영상틀어놓고 댓글 둘러보다가 빵터짐ㅋㅋㅋㅋㅋ존웃,71
4,서예지,kalcls아미틴ㄴ 나자나,15
...,...,...,...
10926,사이다,응아니야,0
10927,그저 그런 사람%,@사이다 응...미안,0
10928,내가어쩌다방시혁이랑취향이같아서,나만 그런지 모르겠는데ㄹㅇ 답정너 개싫음,0
10929,{ENTP} Alice',나도 결론적으로 한국이 싫어..그래서 유학 준비중임 ㅇㅇ,0





100%|██████████| 2/2 [1:42:11<00:00, 3065.63s/it]


In [ ]:
infp = pd.read_csv(BASE_DIR / "full_data"/"INFP_.csv")

In [ ]:
infp.iloc[7:22]

,ID,Comment,Likes
7,놕놕,한 번씩 외향적인 인간처럼 나대다가 후회한 적 개많음,1.4K
8,시켜줘이동혁명예발닦개,와 이거 ㄹㅇㄹㅇㄹㅇㄹㅇ 내 흑역사 중 하나는 그것때문에 생김;;,136
9,새뫠애크 200,이거 진짜임ㅠㅠ소심한 성격 고치려고 막 개그같은것도 찾아보고 하면서 외향적인 사람처...,139
10,55DOBAL,@새뫠애크 200 과거의 일 생각하면서 이불킥 ㅋㅋㅋㅋㅋㅋㅋ,12
11,ᄋᄋ,내가 오늘 그럼...아 겁나 후회되고 쪽팔려,17
12,능덕,ㄴ..나는 외향적인데 낯가려서 평소에 내ㅗㅑㅇ형처럼하는 인프피인데...,6
13,뫌크튜,ㅋㅋㅋㅋㅋㅋㅋㅋㅋ,1
14,Me어켓,격공,1
15,널 보면 재채기가 나올 것 같아,장기자랑때 아재개그 같은고 해서 망했었던 기억이...,4
16,장훈님하고싶은거다해,내 인생 흑역사 TOP1이 인싸 무리 기어들어간거,9


In [ ]:
import pandas as pd
print("pandas version: ", pd.__version__)
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

pandas version:  1.1.5
